# Scpy, a single-cell gene differential expression analysis tool based on pytorch

We use the data of Gasperini et al. [1] as an example to show how to use scpy for differential expression analysis, and compare it with the R package monocle3 [2,3].

Gasperini et al. performed CRISPRi screening in the K562 cell line. In the at-scale experiment, they transfected 6,542 gRNA groups, and measured their effect on 13,135 highly expressed genes in 207,324 cells with single-cell RNA-seq.

We downloaded the expression matrix, gRNA list and phenotype data of the at-scale experiment from GEO (https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE120861).

First, we execute the R workflow.

## Monocle3

In [1]:
library(monocle3)

Loading required package: Biobase

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Welcome to Bioconductor

    Vignettes contain introductory material; view with


In [2]:
base <- '/home/jjpeng/data/yxlong/CRISPR/source/30849375/GEO/'

### prepare featureData & phenoData

In [18]:
featureData <- read.table(gzfile(paste(base, 'GSE120861_at_scale_screen.genes.txt.gz', sep='')), row.names=1)
featureData['id'] <- row.names(featureData)

write.table(featureData, paste(base, 'at_scale.featureData.tsv', sep=''), sep='\t')

phenoData <- read.table(gzfile(paste(base, 'GSE120861_at_scale_screen.phenoData.txt.gz', sep='')), row.names=2)
phenoData <- phenoData[c(1:4, 6:10, 14, 17)]
colnames(phenoData) <- c('sample','total_umis','Size_Factor','gRNA_groups','barcodes','read_count','umi_count','proportion','guide_count','prep_batch','percent.mito')

write.table(phenoData, paste(base, 'at_scale.phenoData.tsv', sep=''), sep='\t')

### load CellDataSet

In [3]:
cds <- monocle3::load_mm_data(
    mat_path = paste(base, 'GSE120861_at_scale_screen.exprs.mtx.gz', sep=''),
    feature_anno_path = paste(base, 'at_scale.featureData.tsv', sep=''),
    cell_anno_path = paste(base, 'at_scale.phenoData.tsv', sep=''),
    header=TRUE
)

cds

Warning message in new_cell_data_set(mat, cell_metadata = cell_annotations$metadata, :
“Warning: gene_metadata must contain a column verbatim named 'gene_short_name' for certain functions.”


class: cell_data_set 
dim: 13135 207324 
metadata(1): cds_version
assays(1): counts
rownames(13135): ENSG00000238009 ENSG00000237683 ... ENSG00000215615
  ENSG00000215699
rowData names(1): id
colnames(207324): AAACCTGAGAGGTACC-1_1A_1_SI-GA-E2
  AAACCTGAGTCAATAG-1_1A_1_SI-GA-E2 ... TTTGTCAGTTCTGTTT-1_2B_8_SI-GA-H9
  TTTGTCATCAAAGTAG-1_2B_8_SI-GA-H9
colData names(12): sample total_umis ... percent.mito n.umi
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):

### choose bassik_mch, a non-targeting control gRNA group and 100 genes as an example

In [4]:
genes = fData(cds)[1:100, 'id']

cds_subset <- cds[rowData(cds)$id %in% genes, ]

pData(cds_subset)['gRNA_group'] <- as.integer(lapply(pData(cds)[,'gRNA_groups'], function(x) grepl('bassik_mch', x)))

cds_subset

class: cell_data_set 
dim: 100 207324 
metadata(1): cds_version
assays(1): counts
rownames(100): ENSG00000238009 ENSG00000237683 ... ENSG00000116285
  ENSG00000142599
rowData names(1): id
colnames(207324): AAACCTGAGAGGTACC-1_1A_1_SI-GA-E2
  AAACCTGAGTCAATAG-1_1A_1_SI-GA-E2 ... TTTGTCAGTTCTGTTT-1_2B_8_SI-GA-H9
  TTTGTCATCAAAGTAG-1_2B_8_SI-GA-H9
colData names(13): sample total_umis ... n.umi gRNA_group
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):

### differential expression analysis

In [5]:
formula <- "~gRNA_group+guide_count+percent.mito+prep_batch"

tick <- Sys.time()
fits <- monocle3::fit_models(cds_subset, expression_family = 'negbinomial', model_formula_str = formula)
print(Sys.time() - tick)

coef <- coefficient_table(fits)[c('gene_id','status','term','estimate','std_err','test_val','p_value','normalized_effect','q_value')]

write.csv(coef, 'bassik_mch-100.csv', row.names=FALSE)

coef

Time difference of 4.225742 mins


gene_id,status,term,estimate,std_err,test_val,p_value,normalized_effect,q_value
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000238009,OK,(Intercept),-4.488624e+00,0.0588449363,-76.27885526,0.000000e+00,0.000000e+00,0.000000e+00
ENSG00000238009,OK,gRNA_group,9.159409e-02,0.3206558563,0.28564609,7.751492e-01,7.142159e-02,1.000000e+00
ENSG00000238009,OK,guide_count,-4.850636e-04,0.0011927896,-0.40666317,6.842554e-01,-3.702237e-04,1.000000e+00
ENSG00000238009,OK,percent.mito,1.176492e+01,0.9931642069,11.84589335,2.259971e-32,1.605482e+01,8.361894e-31
ENSG00000238009,OK,prep_batchprep_batch_2,3.846924e-02,0.0358320781,1.07359778,2.830030e-01,2.963068e-02,1.000000e+00
ENSG00000237683,OK,(Intercept),-3.598773e+00,0.0326945585,-110.07252209,0.000000e+00,0.000000e+00,0.000000e+00
ENSG00000237683,OK,gRNA_group,-1.210731e-01,0.1899799610,-0.63729420,5.239332e-01,-1.258034e-01,1.000000e+00
ENSG00000237683,OK,guide_count,5.331953e-03,0.0006333395,8.41879008,3.803923e-17,5.637260e-03,2.929021e-15
ENSG00000237683,OK,percent.mito,1.494807e+01,0.5411345368,27.62357559,5.798296e-168,2.111604e+01,3.305029e-166


It takes 2.53 seconds to test a relationship pair between a set of variables and a gene, on average.

This means that it will take **6.9 years** to complete the test of 85,929,170 pairs of interactions between 6,542 gRNA groups and 13,135 genes ! ! !

In the paper, Gasperini et al. measured the relationship between each gRNA group and surrounding 1 Mbp genes, a total of 629,515 pairs. They also used negbinomial.size family to increase speed by reducing accuracy.

Now, let's move to python.

## Statsmodels

Scpy is deeply inspired by statsmodels [4] and relies on it to solve optimizations.

We first show how to use statsmodels for differential expression analysis.

In [1]:
import os, sys
from tqdm import tqdm
import time

import pandas as pd
import numpy as np

from scipy import stats

import statsmodels as sm
import statsmodels.api

In [2]:
base = '/home/jjpeng/data/yxlong/CRISPR/source/30849375/GEO/'

### load featureData & phenoData

In [3]:
grnas = pd.read_csv(base + 'GSE120861_grna_groups.at_scale.txt.gz', sep = '\t', header=None)[1]
cells = pd.read_csv(base + 'GSE120861_at_scale_screen.cells.txt.gz', sep = '\t', header=None)[0]
genes = pd.read_csv(base + 'GSE120861_at_scale_screen.genes.txt.gz', sep = '\t', header=None)[0]

In [4]:
phenoData = pd.read_csv(base + 'GSE120861_at_scale_screen.phenoData.txt.gz', sep=' ', header=None, usecols=[0,1,2,5,7,8,9,10,14,17])
phenoData.columns = ['sample','cell','total_umis','gRNA_groups','read_count','umi_count','proportion','guide_count','prep_batch','percent.mito']
phenoData.set_index('cell', inplace=True)

phenoData['guide_count'].fillna(0, inplace=True)
phenoData['size_factor'] = phenoData['total_umis'] / np.exp(np.mean(np.log(phenoData['total_umis'])))
phenoData['prep_batch'] = phenoData.eval('prep_batch != "prep_batch_1"').astype(int)

phenoData

,sample,total_umis,gRNA_groups,read_count,umi_count,proportion,guide_count,prep_batch,percent.mito,size_factor
cell,,,,,,,,,,
AAACCTGAGAGGTACC-1_1A_1_SI-GA-E2,1A_1_SI-GA-E2,17572.0,chr10.845_top_two_chr1.11183_top_two_chr1.1129...,14135.0,964.0,0.969819,67.0,0,0.058787,1.084235
AAACCTGAGTCAATAG-1_1A_1_SI-GA-E2,1A_1_SI-GA-E2,8923.0,chr1.12695_top_two_chr11.3294_top_two_chr1.679...,4329.0,293.0,0.844380,26.0,0,0.036087,0.550571
AAACCTGCAAACAACA-1_1A_1_SI-GA-E2,1A_1_SI-GA-E2,14637.0,ALDH1A2_TSS_BRI3_TSS_chr10.1918_top_two_chr10....,12362.0,884.0,0.950538,61.0,0,0.069823,0.903138
AAACCTGCACTTCTGC-1_1A_1_SI-GA-E2,1A_1_SI-GA-E2,22798.0,C16orf91_TSS_chr1.11332_top_two_chr1.1933_top_...,7459.0,544.0,0.939551,39.0,0,0.026187,1.406692
AAACCTGCATGTAGTC-1_1A_1_SI-GA-E2,1A_1_SI-GA-E2,10136.0,chr10.185_top_two_chr10.484_top_two_chr11.4167...,14831.0,1054.0,0.959927,37.0,0,0.007991,0.625416
...,...,...,...,...,...,...,...,...,...,...
TTTGTCAGTACCTACA-1_2B_8_SI-GA-H9,2B_8_SI-GA-H9,17938.0,BRIX1_TSS_chr10.2059_top_two_chr10.350_top_two...,4570.0,394.0,0.856522,31.0,1,0.060598,1.106818
TTTGTCAGTATCACCA-1_2B_8_SI-GA-H9,2B_8_SI-GA-H9,16543.0,chr10.221_top_two_chr11.3853_top_two_chr11.498...,4311.0,387.0,0.861915,33.0,1,0.057789,1.020743
TTTGTCAGTTCAGACT-1_2B_8_SI-GA-H9,2B_8_SI-GA-H9,15009.0,chr12.3400_top_two_chr4.1281_second_two_ID3_TSS,669.0,57.0,0.876923,3.0,1,0.049037,0.926092


### prepare & load expression data

In [5]:
if not os.path.exists(base + 'at_scale.exprs.npy'):

    mtx = pd.read_csv(base + 'GSE120861_at_scale_screen.exprs.mtx.gz', sep = ' ', header=None, skiprows=2, compression=None)
    coo = sp.sparse.coo_matrix((mtx[2], (mtx[0] - 1, mtx[1] - 1)), shape=(genes.shape[0], cells.shape[0]), dtype=np.int16)
    np.save(base + 'at_scale.exprs.npy', coo)
    
else:

    coo = np.load(base + 'at_scale.exprs.npy', allow_pickle=True).item()

In [6]:
exprs = pd.DataFrame.sparse.from_spmatrix(coo.T, index=cells, columns=genes)

exprs

,ENSG00000238009,ENSG00000237683,ENSG00000228463,ENSG00000237094,ENSG00000235373,ENSG00000228327,ENSG00000237491,ENSG00000225880,ENSG00000230368,ENSG00000188976,...,ENSG00000198886,ENSG00000198786,ENSG00000198695,ENSG00000198727,ENSG00000215750,ENSG00000215689,ENSG00000215781,ENSG00000220023,ENSG00000215615,ENSG00000215699
0,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGAGGTACC-1_1A_1_SI-GA-E2,0,1,1,0,0,0,0,0,0,0,...,115,20,5,90,1,0,0,1,0,0
AAACCTGAGTCAATAG-1_1A_1_SI-GA-E2,0,0,0,0,0,0,0,0,0,0,...,45,17,1,36,0,0,0,1,0,0
AAACCTGCAAACAACA-1_1A_1_SI-GA-E2,0,0,1,0,1,0,0,0,0,3,...,156,20,4,80,0,0,0,1,1,0
AAACCTGCACTTCTGC-1_1A_1_SI-GA-E2,0,0,1,1,0,0,0,0,0,1,...,66,24,1,21,0,0,2,0,0,0
AAACCTGCATGTAGTC-1_1A_1_SI-GA-E2,0,0,1,0,0,0,0,0,0,0,...,8,4,0,15,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCAGTACCTACA-1_2B_8_SI-GA-H9,0,0,0,0,0,0,0,0,0,1,...,119,30,6,97,0,0,1,1,0,0
TTTGTCAGTATCACCA-1_2B_8_SI-GA-H9,0,0,0,0,0,0,0,0,0,3,...,113,32,13,60,0,0,1,1,0,1
TTTGTCAGTTCAGACT-1_2B_8_SI-GA-H9,0,1,2,0,0,0,1,0,0,4,...,93,21,2,68,0,0,0,1,0,0


### choose bassik_mch, a non-targeting control gRNA group and 100 genes as an example

In [7]:
phenoData['gRNA_group'] = phenoData['gRNA_groups'].str.contains('bassik_mch', na=False).astype(int)

subset = exprs[exprs.columns[:100]]

### differential expression analysis

In [17]:
# x

x = phenoData[['gRNA_group','guide_count','percent.mito','prep_batch']].values
x = sm.api.add_constant(x, prepend=False)

xname = ['gRNA_group','guide_count','percent.mito','prep_batch','intercept']

# y

size_factor = phenoData['size_factor'].values

Y = np.round(subset.values.T / size_factor)

# fit

summaries = list()
for gene, y in zip(tqdm(subset.columns), Y):
    
    model = sm.discrete.discrete_model.NegativeBinomial(y, x, loglike_method='geometric')
    res = model.fit(disp=False, warn_convergence=False)
    summary = res.summary2(xname=xname).tables[1]
    
    summary['term'] = summary.index
    summary.index = [gene] * summary.shape[0]
    summaries.append(summary)
    
summaries = pd.concat(summaries, axis=0)

summaries

100%|██████████| 100/100 [02:16<00:00,  1.36s/it]


,Coef.,Std.Err.,z,P>|z|,[0.025,0.975],term
ENSG00000238009,0.121739,0.291886,0.417079,6.766204e-01,-0.450346,0.693825,gRNA_group
ENSG00000238009,-0.000337,0.001084,-0.310907,7.558716e-01,-0.002462,0.001788,guide_count
ENSG00000238009,11.765150,0.905019,12.999893,1.225147e-38,9.991345,13.538954,percent.mito
ENSG00000238009,0.041200,0.032899,1.252321,2.104528e-01,-0.023281,0.105681,prep_batch
ENSG00000238009,-4.495644,0.053886,-83.428977,0.000000e+00,-4.601258,-4.390030,intercept
...,...,...,...,...,...,...,...
ENSG00000142599,-0.111505,0.093053,-1.198287,2.308052e-01,-0.293886,0.070877,gRNA_group
ENSG00000142599,-0.000184,0.000310,-0.593772,5.526647e-01,-0.000793,0.000424,guide_count
ENSG00000142599,5.626938,0.263224,21.376973,2.188725e-101,5.111028,6.142848,percent.mito
ENSG00000142599,0.109529,0.009431,11.614294,3.486571e-31,0.091046,0.128013,prep_batch


### compare with monocle3 results

In [18]:
# load monocle3 results
df = pd.read_csv('bassik_mch-100.csv', index_col=0)

# transform into wide-style table

monocle3_results = df.query('term == "gRNA_group"')[['term','estimate','p_value']]
for i in ['(Intercept)','guide_count','percent.mito','prep_batchprep_batch_2']:
    monocle3_results[i] = df.query('term == "%s"' % i)['estimate']
    
results = summaries.query('term == "gRNA_group"')[['term','Coef.','P>|z|']]
for i in ['intercept','guide_count','percent.mito','prep_batch']:
    results[i] = summaries.query('term == "%s"' % i)['Coef.']
    
# Pearson correlation test

print('Spearman correlation of')
print('effect size', *stats.spearmanr(results['Coef.'], monocle3_results['estimate']))
print('intercept', *stats.spearmanr(results['intercept'], monocle3_results['(Intercept)']))
print('p-value', *stats.spearmanr(results['P>|z|'], monocle3_results['p_value']))

Spearman correlation of
effect size 0.9972397239723971 1.7145424785108527e-112
intercept 0.9998799879987997 3.439317197857124e-179
p-value 0.9475907590759075 2.314617210933284e-50


Statsmodels got highly consistent results with monocle3, but it was still too slow.

Now, let's try scpy.

## Scpy

In [11]:
import torch
import scpy

### differential expression analysis

Scpy uses an API similar to statsmodels when fitting a single pair.

In [20]:
summaries = list()
for gene, y in zip(tqdm(subset.columns), Y):
    
    summary = scpy.fit(x, y, 'negative_binomial', device='cpu', xname=xname)
    
    summary['term'] = summary.index
    summary.index = [gene] * summary.shape[0]
    summaries.append(summary)
    
summaries = pd.concat(summaries, axis=0)

summaries

100%|██████████| 100/100 [00:10<00:00,  9.10it/s]


,Coef.,Std.Err.,z,P>|z|,term
ENSG00000238009,0.118145,0.292382,0.404078,6.861557e-01,gRNA_group
ENSG00000238009,-0.000333,0.001084,-0.306772,7.590171e-01,guide_count
ENSG00000238009,11.762953,0.905033,12.997268,1.312820e-38,percent.mito
ENSG00000238009,0.041310,0.032900,1.255632,2.092508e-01,prep_batch
ENSG00000238009,-4.495757,0.053887,-83.429970,0.000000e+00,intercept
...,...,...,...,...,...
ENSG00000142599,-0.113176,0.093100,-1.215645,2.241214e-01,gRNA_group
ENSG00000142599,-0.000190,0.000311,-0.611838,5.406457e-01,guide_count
ENSG00000142599,5.665720,0.263223,21.524412,1.192395e-102,percent.mito
ENSG00000142599,0.109481,0.009431,11.609066,3.789701e-31,prep_batch


In [21]:
results = summaries.query('term == "gRNA_group"')[['term','Coef.','P>|z|']]
for i in ['intercept','guide_count','percent.mito','prep_batch']:
    results[i] = summaries.query('term == "%s"' % i)['Coef.']
    
# Pearson correlation test

print('Spearman correlation of')
print('effect size', *stats.spearmanr(results['Coef.'], monocle3_results['estimate']))
print('intercept', *stats.spearmanr(results['intercept'], monocle3_results['(Intercept)']))
print('p-value', *stats.spearmanr(results['P>|z|'], monocle3_results['p_value']))

Spearman correlation of
effect size 0.9985718571857185 1.678441941676762e-126
intercept 0.9998919891989198 1.9700524468233987e-181
p-value 0.9574557455745574 1.071068835741778e-54


Wow, it's **20X faster** and as accurate as monocle3.

### differential expression analysis in batch

Scpy provides a more convenient and efficient API for fitting multiple sets of independent variables and dependent variables at the same time.

In [14]:
summaries = scpy.fit_batch([x], Y, 'negative_binomial', device='cpu', xnames=[('bassik_mch', xname)], yname=subset.columns)

summaries

100%|██████████| 100/100 [00:05<00:00, 16.95it/s]


Coef.  Std.Err.           z  \
group      term         target                                             
bassik_mch gRNA_group   ENSG00000238009   0.118145  0.292382    0.404078   
           guide_count  ENSG00000238009  -0.000333  0.001084   -0.306772   
           percent.mito ENSG00000238009  11.762953  0.905033   12.997268   
           prep_batch   ENSG00000238009   0.041310  0.032900    1.255632   
           intercept    ENSG00000238009  -4.495757  0.053887  -83.429970   
...                                            ...       ...         ...   
           gRNA_group   ENSG00000142599  -0.113176  0.093100   -1.215645   
           guide_count  ENSG00000142599  -0.000190  0.000311   -0.611838   
           percent.mito ENSG00000142599   5.665720  0.263223   21.524412   
           prep_batch   ENSG00000142599   0.109481  0.009431   11.609066   
           intercept    ENSG00000142599  -1.550258  0.015097 -102.684689   

                                                 P>|z|  
group      term         target                          
bassik_mch gRNA_group   ENSG00000238009   6.861557e-01  
           guide_count  ENSG00000238009   7.590171e-01  
           percent.mito ENSG00000238009   1.312820e-38  
           prep_batch   ENSG00000238009   2.092508e-01  
           intercept    ENSG00000238009   0.000000e+00  
...                                                ...  
           gRNA_group   ENSG00000142599   2.241214e-01  
           guide_count  ENSG00000142599   5.406457e-01  
           percent.mito ENSG00000142599  1.192395e-102  
           prep_batch   ENSG00000142599   3.789701e-31  
           intercept    ENSG00000142599   0.000000e+00  

[500 rows x 4 columns]

If you have a GPU, scpy can run faster.

In [15]:
summaries = scpy.fit_batch([x], Y, 'negative_binomial', device=0, xnames=[('bassik_mch', xname)], yname=subset.columns)

100%|██████████| 100/100 [00:01<00:00, 54.09it/s]


Sometimes, the size of the dependent variable is large, or it takes a long time for preprocessing.

For example, the expression matrix in the sample occupies 6.5 GB of memory when stored in a sparse form. If it is converted to dense form all at once for calculation, it will occupy at least 30GB of memory.

Scpy provides lazy data loader to solve this problem.

In [16]:
loader = lambda i: np.round(subset[subset.columns[i]].values.T / size_factor)
Ys = scpy.dataloader(loader, len(subset.columns), num_workers=2)

summaries = scpy.fit_batch([x], Ys, 'negative_binomial', device=0, xnames=[('bassik_mch', xname)], yname=subset.columns)

100%|██████████| 100/100 [00:02<00:00, 47.75it/s]


So far, we have been able to obtain **100X faster** than the R package.

Can it be faster?

### differential expression analysis in parallel

In order to maximize the performance of scpy and the device, you can use the parallel API.

Parallel scpy only provides a command line API based on multiprocessing, and requires a python script to export args.

We provide a sample script, *demo.py* .

```python
# prepare

## x

# select 2 gRNA groups targeting non-coding regions, 2 positiv control gRNA groups and 3 non-targeting control gRNA
gRNA_groups = ['chr1.8492','chrX.952','CERS2_TSS','GATA2_TSS','bassik_mch','random_1','scrambled_1']

X, xnames = list(), list()

for gRNA_group in gRNA_groups:
  
  geno = phenoData['gRNA_groups'].str.contains(gRNA_group, na=False) # get genotypes
  
  x = geno.astype(int).values.reshape((-1, 1)) # use genotype as x
  cov = phenoData[['guide_count','percent.mito','prep_batch']].values # covariants
  c = np.ones((x.shape[0], 1)) # constraint

  X.append(np.concatenate([x, cov, c], axis=1))
  xnames.append((gRNA_group, ['gRNA_group', 'guide_count', 'percent.mito', 'prep_batch', 'intercept']))
  
## y

size_factor = phenoData['Size_Factor'].values # load size factor calculate by monocle

loader = lambda i: np.round(exprs[exprs.columns[i]].values.T / size_factor)


# export

args = {
  'X': X,
  'Y': {'data': loader, 'length': exprs.shape[1]},
  'family': 'negative_binomial',
  'xnames': xnames,
  'yname': exprs.columns
}
```

If I have 8 GPUs, I can run the following commands in the console:

```bash
python -m scpy demo.py demo.csv -d 0,1,2,3,4,5,6,7 --disp=True --low_memory=True
```

```
100%|█████████████████████████████████████████████████████████████████████████████████| 1642/1642 [03:31<00:00,  7.77it/s]
100%|█████████████████████████████████████████████████████████████████████████████████| 1642/1642 [03:31<00:00,  7.77it/s]
100%|█████████████████████████████████████████████████████████████████████████████████| 1642/1642 [03:32<00:00,  7.72it/s]
100%|█████████████████████████████████████████████████████████████████████████████████| 1642/1642 [03:33<00:00,  7.69it/s]
100%|█████████████████████████████████████████████████████████████████████████████████| 1642/1642 [03:32<00:00,  7.71it/s]
100%|█████████████████████████████████████████████████████████████████████████████████| 1641/1641 [03:33<00:00,  7.68it/s]
100%|█████████████████████████████████████████████████████████████████████████████████| 1642/1642 [03:33<00:00,  7.69it/s]
100%|█████████████████████████████████████████████████████████████████████████████████| 1642/1642 [03:34<00:00,  7.66it/s]
221.72746539115906
                                                    Coef.  Std.Err.          z         P>|z|
group             term         target
chr1.8492_top_two gRNA_group   ENSG00000238009   0.202458  0.196002   1.032938  3.016340e-01
                  guide_count  ENSG00000238009   0.008593  0.001002   8.579515  9.591534e-18
                  percent.mito ENSG00000238009  17.562862  0.873383  20.109011  7.500587e-90
                  prep_batch   ENSG00000238009  -0.019360  0.031808  -0.608657  5.427524e-01
                  intercept    ENSG00000238009  -4.929867  0.053910 -91.446486  0.000000e+00
...                                                   ...       ...        ...           ...
bassik_mch        gRNA_group   ENSG00000215699   0.388362  0.320037   1.213490  2.249438e-01
                  guide_count  ENSG00000215699   0.006148  0.001344   4.574964  4.765763e-06
                  percent.mito ENSG00000215699  13.970382  1.161668  12.026145  2.656786e-33
                  prep_batch   ENSG00000215699   0.043774  0.042161   1.038266  2.991475e-01
                  intercept    ENSG00000215699  -5.298037  0.070448 -75.204969  0.000000e+00

[459725 rows x 4 columns]
```

**OMG, we are now 1000X faster ! ! !**

Let us check whether the results of scpy are consistent with those provided by Gasperini et al.

In [3]:
# load results from the paper

paper = pd.read_csv(base + 'GSE120861_all_deg_results.at_scale.txt.gz', sep='\t', index_col='pairs4merge')

paper

/home/jjpeng/data/.conda/envs/torch-latest/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,beta,intercept,fold_change.transcript_remaining,pvalue.raw,pvalue.empirical,pvalue.empirical.adjusted,ENSG,gene_short_name,gRNA_group,quality_rank_grna,target_site.chr,target_site.start,target_site.stop,target_gene.chr,target_gene.start,target_gene.stop,strand,site_type,outlier_gene
pairs4merge,,,,,,,,,,,,,,,,,,,
ACTB_TSS:ENSG00000008256,-0.114784,-3.432141,0.891559,2.746239e-01,0.261969034,0.831159512731444,ENSG00000008256,CYTH3,ACTB_TSS,control,chr7,5570339,5570340,chr7,6312274,6312275,-,TSS,False
ACTB_TSS:ENSG00000011275,-0.042450,-1.464958,0.958438,2.771529e-01,0.264361067,0.831814624048016,ENSG00000011275,RNF216,ACTB_TSS,control,chr7,5570339,5570340,chr7,5821369,5821370,-,TSS,False
ACTB_TSS:ENSG00000075618,0.034076,0.392365,1.034664,1.425945e-01,not_applicable,not_applicable,ENSG00000075618,FSCN1,ACTB_TSS,control,chr7,5570339,5570340,chr7,5632454,5632455,+,TSS,False
ACTB_TSS:ENSG00000075624,-0.437939,2.898258,0.645365,7.463645e-115,3.941e-06,0.000407265538461538,ENSG00000075624,ACTB,ACTB_TSS,control,chr7,5570339,5570340,chr7,5570339,5570340,-,selfTSS,False
ACTB_TSS:ENSG00000086232,-0.010209,1.154783,0.989843,6.253678e-01,0.611785198,0.947813794532952,ENSG00000086232,EIF2AK1,ACTB_TSS,control,chr7,5570339,5570340,chr7,6098860,6098861,-,TSS,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZRANB2_TSS:ENSG00000116754,-0.016496,0.174595,0.983639,4.979588e-01,0.482804551,0.916882597262894,ENSG00000116754,SRSF11,ZRANB2_TSS,control,chr1,71546971,71546972,chr1,70671365,70671366,+,TSS,False
ZRANB2_TSS:ENSG00000116761,0.022505,-1.795345,1.022760,7.781657e-01,not_applicable,not_applicable,ENSG00000116761,CTH,ZRANB2_TSS,control,chr1,71546971,71546972,chr1,70876901,70876902,+,TSS,False
ZRANB2_TSS:ENSG00000118454,-0.039160,-2.665878,0.961597,6.191715e-01,0.605440595,0.946588425199907,ENSG00000118454,ANKRD13C,ZRANB2_TSS,control,chr1,71546971,71546972,chr1,70820416,70820417,-,TSS,False


In [7]:
df = pd.read_csv('demo.csv')

for gRNA_group, g in df.groupby('group'):
    
    g.set_index('target', inplace=True)
    
    results = g.query('term == "gRNA_group"')[['term','Coef.','P>|z|']]
    results['intercept'] = g.query('term == "intercept"')['Coef.']
    for i in ['guide_count','percent.mito','prep_batch']:
        results[i] = g.query('term == "%s"' % i)['Coef.']
        
    paper_results = paper.query('gRNA_group == "%s"' % gRNA_group)
    paper_results.set_index('ENSG', inplace=True)
    paper_results = paper_results.loc[paper_results.index.intersection(results.index)]
    
    results = results.loc[paper_results.index]
    
    print('\n', gRNA_group, '\n')

    print('effect size', *stats.spearmanr(results['Coef.'], paper_results['beta']))
    print('intercept', *stats.spearmanr(results['intercept'], paper_results['intercept']))
    print('p-value', *stats.spearmanr(results['P>|z|'], paper_results['pvalue.raw']))


 CERS2_TSS 

effect size 0.9823353050806256 9.800019237294788e-31
intercept 0.998541447208492 2.4632354748867975e-52
p-value 0.95753990762499 3.235472204989662e-23

 GATA2_TSS 

effect size 0.9964912280701754 9.285165080058482e-20
intercept 0.9982456140350877 2.579846712328762e-22
p-value 0.9210526315789472 2.2361332893791837e-08

 bassik_mch 

effect size 0.9879195444074778 0.0
intercept 0.9994431151886197 0.0
p-value 0.9590769458326537 0.0

 chr1.8492_second_two 

effect size 0.9931934203062961 5.631027405850918e-39
intercept 0.9987035086297708 2.339336383494484e-53
p-value 0.9530021878291874 2.3641247234831865e-22

 chr1.8492_top_two 

effect size 0.9701806984847258 3.095784714315404e-26
intercept 0.9987035086297708 2.339336383494484e-53
p-value 0.9701806984847258 3.095784714315404e-26

 chrX.952_second_two 

effect size 0.9952346041055717 6.809255742590661e-32
intercept 0.9978005865102637 6.361252877369035e-37
p-value 0.9754398826979471 2.8726637865941426e-21

 chrX.952_top_two 



## Reference

[1] Gasperini M, Hill AJ, McFaline-Figueroa JL, Martin B et al. A Genome-wide Framework for Mapping Gene Regulation via Cellular Genetic Screens. Cell 2019 Jan 10;176(1-2):377-390.e19. PMID: 30612741

[2] Qiu, X. et. al. Reversed graph embedding resolves complex single-cell trajectories. Nat. Methods 14, 979–982 (2017). https://doi.org/10.1038/nmeth.4402

[3] Cao, J. et. al. The single-cell transcriptional landscape of mammalian organogenesis. Nature 566, 496–502 (2019). https://doi.org/10.1038/s41586-019-0969-x

[4] Seabold, Skipper, and Josef Perktold. “statsmodels: Econometric and statistical modeling with python.” Proceedings of the 9th Python in Science Conference. 2010.